In [ ]:
!pip install python-docx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 17.3 MB/s eta 0:00:00


In [ ]:
!pip install PyMuPDF


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 54.9 MB/s eta 0:00:00


In [ ]:
from sentence_transformers import SentenceTransformer, util
import fitz  # PyMuPDF for PDFs
import numpy as np
import re
import requests

# Initialize Sentence-BERT model
model = SentenceTransformer('all-MiniLM-L6-v2')

# NVIDIA API details
NVIDIA_API_URL = "https://integrate.api.nvidia.com/v1/chat/completions"
API_KEY = "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"  # Replace with your valid NVIDIA API key

# Function to load and parse Q&A from PDF
def extract_qa_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = " ".join([page.get_text() for page in doc])

    # Extract structured Q&A pairs using regex
    qa_pairs = []
    qa_pattern = r'\"question\":\s*\"(.*?)\",\s*\"answer\":\s*\"(.*?)\"'
    matches = re.findall(qa_pattern, text, re.DOTALL)

    for match in matches:
        question, answer = match
        qa_pairs.append({"question": question.strip(), "answer": answer.strip()})

    return qa_pairs

# Retrieve best-matching answer
def get_best_answer(user_question, qa_pairs):
    questions = [qa["question"] for qa in qa_pairs]
    question_embeddings = model.encode(questions, convert_to_tensor=True)
    user_embedding = model.encode(user_question, convert_to_tensor=True)

    # Compute similarity scores
    scores = util.cos_sim(user_embedding, question_embeddings).cpu().numpy()
    best_match_idx = np.argmax(scores)
    best_match_score = scores[0][best_match_idx]

    # Set a similarity threshold to determine if it's a valid match
    if best_match_score > 0.75:
        return qa_pairs[best_match_idx]["answer"]
    else:
        return None  # No exact match found

# NVIDIA API for additional responses
def generate_answer_via_nvidia(question):
    headers = {"Authorization": f"Bearer {API_KEY}"}
    payload = {
        "model": "microsoft/phi-3-mini-128k-instruct",
        "messages": [{"role": "user", "content": question}],
        "temperature": 0.2,
        "top_p": 0.8,
        "max_tokens": 300,
    }

    response = requests.post(NVIDIA_API_URL, json=payload, headers=headers)
    response_data = response.json()

    if response.status_code == 200 and "choices" in response_data:
        return response_data["choices"][0]["message"]["content"].strip()
    else:
        return "I'm unable to find an answer."

# Main function with a limit of 5 questions
def main(pdf_path):
    qa_pairs = extract_qa_from_pdf(pdf_path)
    question_count = 0  # Counter for questions

    while question_count < 5:  # Limit to 5 questions
        user_question = input(f"({question_count+1}/5) Enter your question (or type 'exit' to quit): ")
        if user_question.lower() == "exit":
            break

        answer = get_best_answer(user_question, qa_pairs)

        if answer:
            print(f"\n✅ Answer: {answer}")
        else:
            print("\n❌ Exact match not found. Generating a response...")
            ai_answer = generate_answer_via_nvidia(user_question)
            print(f"\n🤖 AI-Generated Answer: {ai_answer}")

        question_count += 1  # Increment question count

    print("\n🎯 You have reached the limit of 5 questions. Exiting...")

# Run with your PDF
pdf_path = "/content/Campus_Pal Content.pdf"
main(pdf_path)


(1/5) Enter your question (or type 'exit' to quit): Who is the founder and Chancellor of Amrita Vishwa Vidyapeetham? 

✅ Answer: Sri Mata Amritanandamayi Devi, affectionately known as 
AMMA, is the founder and Chancellor of the university.
(2/5) Enter your question (or type 'exit' to quit): How many campuses does Amrita Vishwa Vidyapeetham have, and can  you list them? 

✅ Answer: The university has multiple campuses, including 
Amritapuri, Amaravati, Bengaluru, Chennai, Coimbatore, Kochi, Mysuru, and 
Faridabad.
(3/5) Enter your question (or type 'exit' to quit): What is the size of the Amritapuri campus at Amrita Vishwa  Vidyapeetham? 

✅ Answer: The campus spans over 80 acres of land.
(4/5) Enter your question (or type 'exit' to quit): What is the tuition fee per semester for the B.B.A. (Business Analytics)  program for the academic year 2025-26? 

✅ Answer: Rs. 45,900.
(5/5) Enter your question (or type 'exit' to quit): What is the NAAC accreditation grade for Amrita Vishwa Vidyape